# Section 2: Analyzing Causal Forests on Simulated Data similar to GSS data 

In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from econml.dml import CausalForestDML
from tqdm.auto import tqdm

from copy import deepcopy
import warnings
warnings.filterwarnings("ignore")

def fullDisplay():
    pd.set_option("display.max_rows", None, "display.max_columns", None)

def defaultDisplay():
    pd.reset_option('^display.', silent=True)

## Loading data

In [2]:
welfare = pd.read_csv("Data/welfare_clean.csv", low_memory=False)
treatments = welfare['w']
labels = welfare['y']
welfare.drop(columns=['w', 'y'], inplace=True)
welfare

,year,id,wrkstat,hrs1,hrs2,evwork,occ,prestige,wrkslf,wrkgovt,...,adults_miss,unrelat_miss,earnrs_miss,income_miss,rincome_miss,income86_miss,partyid_miss,polviews_miss,attblack,attblack_miss
0,0,1,7,0.004845,0.005228,1,135,0.005641,2,2,...,0,0,0,0,0,0,0,0,0.005440,0
1,0,2,1,0.005055,0.005228,0,106,0.006538,2,2,...,0,1,0,0,1,0,0,0,0.004080,0
2,0,3,7,0.004845,0.005228,1,99,0.006538,2,2,...,0,1,0,0,0,0,0,0,0.002040,0
3,0,4,3,0.005055,0.005228,0,142,0.004615,2,0,...,0,0,0,0,1,0,0,0,0.004080,0
4,0,5,8,0.005055,0.005228,1,211,0.005171,2,1,...,0,0,0,0,0,0,0,0,0.004080,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36496,15,2040,3,0.005055,0.005228,0,211,0.005171,2,2,...,0,0,0,0,1,1,0,0,0.004080,0
36497,15,2041,3,0.005055,0.005228,0,211,0.005171,2,2,...,0,0,0,0,1,1,0,0,0.006120,0
36498,15,2042,7,0.004845,0.005228,1,211,0.005171,2,2,...,0,1,0,0,0,1,0,0,0.004080,0
36499,15,2043,7,0.005935,0.005228,1,211,0.005171,2,2,...,0,1,0,0,1,1,0,0,0.005021,1


## DGP, Estimation, and Evaluation functions

In [14]:
def dgp(welfare, effect_type="heterogeneous", effect_homogeneous=10, effect_heterogeneous=2,
        treatment_type="binary", treatment_probability=0.5, heterogeneous_select=4, overlap=True,
        overlap_percent=0.5, order=3, linearity="med", N=5000):    
    
    featureNames = list(welfare.columns)

    # Define non-confounders
    importantFeatureNames = ['wrkstat', 'race', 'year', 'hrs1', 'income', 'occ80', 'id', 'educ'] # top 8 important features based on Shapley visualization from 
    importantFeatureIndices = []
    for name in importantFeatureNames:
        importantFeatureIndices.append(featureNames.index(name)) 

    # Error terms
    error = np.random.normal(size=(N,1))

    # Data generation
    cov = welfare.cov()
    means = welfare.mean(axis=0)
    X = np.random.multivariate_normal(means.values, cov, size=N, check_valid='warn', tol=1e-8)

    # Linearity specification
    if linearity != 'full':
        select = 0 # select n most important features for interactions and polynomials
        poly = PolynomialFeatures(degree=order, interaction_only=False, include_bias=False)

        if linearity == "high": 
            select = 2
        elif linearity == "med": 
            select = 4
        elif linearity == "low": 
            select = 8
        else: # if some typo, assume baseline of high
            select = 2

        poly.fit(X[:, importantFeatureIndices[:select]])
        fullData = poly.transform(X[:, importantFeatureIndices[:select]])
        fullNames = poly.get_feature_names(input_features=importantFeatureNames[:select])
        higherData = fullData[:, -(fullData.shape[1] - select):] # select only higher order
        higherNames = fullNames[-(len(fullNames) - select):]

        X = np.append(X, higherData, axis=1) 
        featureNames.extend(list(higherNames))

    # Treatment selection
    if treatment_type == "binary":
        # randomly assigned treatments with propensity treatment_probability
        treatments = np.random.choice([0, 1], size=N, p=[1 - treatment_probability, treatment_probability]).reshape((-1, 1))
        if not overlap:
            forced = random.sample(list(np.arange(treatments.shape[0])), int(treatments.shape[0] * overlap_percent))
            treatments[forced] = 0
        treated = treatments > 0

        # generate counterfactual treatment indicator vector
        c_treatments = deepcopy(treatments)
        c_treatments[treatments == 0] = 1
        c_treatments[treatments == 1] = 0
    else:
        treatments = np.random.uniform(size=(N, 1))
        if not overlap:
            forced = random.sample(list(np.arange(treatments.shape[0])), int(treatments.shape[0] * overlap_percent))
            treatments[forced] = 0
        treated = treatments > 0.5

        # to-do: counterfactual treatment vector
        c_treatments = deepcopy(treatments)
        c_treatments[treatments > 0] = 0
        c_treatments[treatments == 0] = treatments.mean()

    # Treatment effect calculation
    if effect_type == "homogeneous":

        T = treatments*effect_homogeneous
        CT = c_treatments*effect_homogeneous

    else: 
        # heterogeneous treatment is effect 1 + effect * (sum of first heterogeneous_select important variables)
        heterogeneousIndices = importantFeatureIndices[:heterogeneous_select]

        T = 1 + (effect_heterogeneous*(X[:, heterogeneousIndices].sum(axis=1)))*treatments.ravel()
        T = T.reshape(-1, 1)
        T[~treated] = np.zeros((~treated).sum())
        
        # counterfactual effect
        CT = 1 + (effect_heterogeneous*(X[:, heterogeneousIndices].sum(axis=1)))*c_treatments.ravel()
        CT = CT.reshape(-1, 1)
        CT[treated] = np.zeros((treated).sum()) # everyone that was treated goes to untreated in counterfactual

    # Outcome calculation
    betas = np.random.normal(size=X.shape[1]).reshape(-1,1)
    y = T + X@betas + error
    cy = CT + X@betas + error
    
    # ATE calculation
    empirical_treated = y[treated] - cy[treated] # the treated (in y)
    empirical_untreated = cy[~treated] - y[~treated] # the untreated
    ate = (empirical_treated.mean() + empirical_untreated.mean())/2
    
    return y, X, betas, featureNames, treatments, cy, ate

def estimate_cf(y, X, treatments, test_size=0.2, criterion='het', cv=5, discrete_treatment=True):
    # split data into train and test sets 
    X_train, X_test, Y_train, Y_test, T_train, T_test = train_test_split(X, y, treatments, test_size=test_size)
        
    # specify hyperparams of model
    est = CausalForestDML(criterion=criterion, 
                            n_estimators=1000,       
                            max_samples=0.5,
                            discrete_treatment=discrete_treatment,
                            honest=True,
                            inference=True,
                            cv=cv)
    # fit model
    est.fit(Y_train, T_train, X=X_train, W=None)
        
    return est, X_test

In [48]:
# compute evaluation stats over K simulations
def evalStats(estimators, trueATES, evalSets):
    K = len(estimators)

    lbs = np.zeros(K)
    ubs = np.zeros(K)
    ates = np.zeros(K)
    
    totalBias = 0
    totalRMSE = 0
    totalCoverage = 0
    totalInterval = 0

    for i in range(K):
        estimateATE = estimators[i].ate(evalSets[i])[0]
        lb, ub = estimators[i].ate_interval(evalSets[i], alpha=0.05)
        lbs[i] = lb
        ubs[i] = ub
        ates[i] = estimateATE
        
        bias = trueATES[i] - estimateATE
        totalBias += bias
        
        rmse = bias**2
        totalRMSE += rmse

        coverage = 1 if (trueATES[i] >= lb and trueATES[i] <= ub) else 0
  
        totalCoverage += coverage

        interval = ub - lb
        totalInterval += interval
    
    return totalBias / K, (totalRMSE / K)**0.5, totalCoverage / K, totalInterval / K, lbs, ubs, ates

## (1) Sample Size

In [51]:
K = 10
Ns = [1000, 5000, 10000]

biasNs = np.zeros(len(Ns))
rmseNs = np.zeros(len(Ns))
coverageNs = np.zeros(len(Ns))
intervalNs = np.zeros(len(Ns))
lbsNs = np.zeros((len(Ns), K))
ubsNs = np.zeros((len(Ns), K))
atesNs = np.zeros((len(Ns), K))


for idx, N in enumerate(Ns):
    estNs = []
    trueATENs = []
    evalSetNs = []
    for i in tqdm(range(K)):
        y, X, trueBetas, featureNames, treatments, cy, ate = dgp(welfare, N=N)
        est, X_test = estimate_cf(y, X, treatments)

        estNs.append(est)
        trueATENs.append(ate)
        evalSetNs.append(X_test)

    bias, rmse, coverage, interval, lbs, ubs, ates = evalStats(estNs, trueATENs, evalSetNs)
    
    biasNs[idx] = bias
    rmseNs[idx] = rmse
    coverageNs[idx] = coverage
    intervalNs[idx] = interval
    lbsNs[idx, :] = lbs
    ubsNs[idx, :] = ubs
    atesNs[idx, :] = ates

In [52]:
biasNs

array([1.3866442 , 0.41393999, 0.26439501])

In [53]:
rmseNs

array([1.80910001, 0.71593782, 0.44949236])

In [54]:
coverageNs

array([1., 1., 1.])

In [55]:
intervalNs

array([29.95564525, 30.811189  , 25.37124169])

In [59]:
(ubsNs - lbsNs).mean(axis=1)

array([29.95564525, 30.811189  , 25.37124169])

In [61]:
trueATENs

In [62]:
ubs

array([41.6324963 , 52.05796974, 48.70342181, 45.56344845, 45.90192798,
       39.86938087, 37.45888749, 42.19168884, 46.49296029, 47.76741276])

In [63]:
lbs

array([23.02466055, 11.333274  , 15.53626044, 18.25979735, 17.57188093,
       24.58342774, 26.52464077, 23.14815783, 18.25019432, 15.69488366])

In [57]:
(ubsNs - lbsNs)

array([[41.5312077 , 48.77592907, 27.11900139, 27.13918625, 16.21405757,
        23.40580884, 22.76373827, 21.0484908 , 23.39983298, 48.15919964],
       [33.57961676, 36.67235582, 15.38704597, 33.54752342, 42.3600772 ,
        32.69043489, 29.295068  , 19.76962533, 35.0395684 , 29.77057425],
       [18.60783575, 40.72469574, 33.16716137, 27.30365109, 28.33004705,
        15.28595313, 10.93424672, 19.04353102, 28.24276598, 32.0725291 ]])

In [26]:
import joblib

joblib.dump(biasNs, 'Results/biasNs.pkl')
joblib.dump(rmseNs, 'Results/rmseNs.pkl')
joblib.dump(coverageNs, 'Results/coverageNs.pkl')
joblib.dump(intervalNs, 'Results/intervalNs.pkl')

['Results/intervalNs.pkl']

In [ ]:
biasNs = joblib.load('Results/biasNs.pkl')
rmseNs = joblib.load('Results/rmseNs.pkl')
coverageNs = joblib.load('Results/coverageNs.pkl')
intervalNs = joblib.load('Results/intervalNs.pkl')

## (2) Degree of non-linearity in X

In [23]:
K = 10
linearityLevels = ['full', 'high', 'med', 'low']

biasLLs = np.zeros(len(linearityLevels))
rmseLLs = np.zeros(len(linearityLevels))
coverageLLs = np.zeros(len(linearityLevels))
intervalLLs = np.zeros(len(linearityLevels))

for idx, level in enumerate(linearityLevels):
    estLLs = []
    trueATELLs = []
    evalSetLLs = []
    for i in tqdm(range(K)):
        y, X, trueBetas, featureNames, treatments, cy, ate = dgp(welfare, linearity=level)
        est, X_test = estimate_cf(y, X, treatments)

        estLLs.append(est)
        trueATELLs.append(ate)
        evalSetLLs.append(X_test)

    bias, rmse, coverage, interval, lbs, ubs, ates = evalStats(estLLs, trueATELLs, evalSetLLs)
    
    biasLLs[idx] = bias
    rmseLLs[idx] = rmse
    coverageLLs[idx] = coverage
    intervalLLs[idx] = interval

In [27]:
import joblib

joblib.dump(biasLLs, 'Results/biasLLs.pkl')
joblib.dump(rmseLLs, 'Results/rmseLLs.pkl')
joblib.dump(coverageLLs, 'Results/coverageLLs.pkl')
joblib.dump(intervalLLs, 'Results/intervalLLs.pkl')

['Results/intervalLLs.pkl']

## (3) Percentage Treated

In [17]:
K = 10
percentTreats = [0.1, 0.5, 0.9]

biasPTs = np.zeros(len(percentTreats))
rmsePTs = np.zeros(len(percentTreats))
coveragePTs = np.zeros(len(percentTreats))
intervalPTs = np.zeros(len(percentTreats))

for idx, percent in enumerate(percentTreats):
    estPTs = []
    trueATEPTs = []
    evalSetPTs = []
    for i in tqdm(range(K)):
        y, X, trueBetas, featureNames, treatments, cy, ate = dgp(welfare, treatment_probability=percent)
        est, X_test = estimate_cf(y, X, treatments)

        estPTs.append(est)
        trueATEPTs.append(ate)
        evalSetPTs.append(X_test)

    bias, rmse, coverage, interval, lbs, ubs, ates = evalStats(estPTs, trueATEPTs, evalSetPTs)
    
    biasPTs[idx] = bias
    rmsePTs[idx] = rmse
    coveragePTs[idx] = coverage
    intervalPTs[idx] = interval

In [18]:
import joblib
joblib.dump(biasPTs, 'Results/biasPTs.pkl')
joblib.dump(rmsePTs, 'Results/rmsePTs.pkl')
joblib.dump(coveragePTs, 'Results/coveragePTs.pkl')
joblib.dump(intervalPTs, 'Results/intervalPTs.pkl')

['Results/intervalPTs.pkl']

In [42]:
biasPTs

array([-2.06515279,  0.38423121,  1.46157868])

In [43]:
rmsePTs

array([4.35001693, 0.45434897, 3.50965091])

In [44]:
coveragePTs

array([1., 1., 1.])

In [47]:
intervalPTs

array([43.02529376, 26.92967731, 33.19245789])

## (4) Overlap

In [10]:
K = 10
overlapTypes = [True, False]

biasOTs = np.zeros(len(overlapTypes))
rmseOTs = np.zeros(len(overlapTypes))
coverageOTs = np.zeros(len(overlapTypes))
intervalOTs = np.zeros(len(overlapTypes))

for idx, overlap in enumerate(overlapTypes):
    estOTs = []
    trueATEOTs = []
    evalSetOTs = []
    for i in tqdm(range(K)):
        y, X, trueBetas, featureNames, treatments, cy, ate = dgp(welfare, overlap=overlap)
        est, X_test = estimate_cf(y, X, treatments)

        estOTs.append(est)
        trueATEOTs.append(ate)
        evalSetOTs.append(X_test)

    bias, rmse, coverage, interval, lbs, ubs, ates = evalStats(estOTs, trueATEOTs, evalSetOTs)
    
    biasOTs[idx] = bias
    rmseOTs[idx] = rmse
    coverageOTs[idx] = coverage
    intervalOTs[idx] = interval

In [19]:
import joblib
joblib.dump(biasOTs, 'Results/biasOTs.pkl')
joblib.dump(rmseOTs, 'Results/rmseOTs.pkl')
joblib.dump(coverageOTs, 'Results/coverageOTs.pkl')
joblib.dump(intervalOTs, 'Results/intervalOTs.pkl')

['Results/intervalOTs.pkl']

In [31]:
biasOTs

array([0.3521126, 0.5753263])

In [32]:
rmseOTs

array([0.66382688, 2.79374585])

In [33]:
coverageOTs

array([1., 1.])

In [34]:
intervalOTs

array([25.99187606, 30.47530874])

## (5) Treatment Effect Heterogeneity Level

In [11]:
K = 10
heterogeneityLevels = [0, 2, 4, 8]

biasHLs = np.zeros(len(heterogeneityLevels))
rmseHLs = np.zeros(len(heterogeneityLevels))
coverageHLs = np.zeros(len(heterogeneityLevels))
intervalHLs = np.zeros(len(heterogeneityLevels))

for idx, heterogeneity in enumerate(heterogeneityLevels):
    estHLs = []
    trueATEHLs = []
    evalSetHLs = []
    for i in tqdm(range(K)):
        if heterogeneity == 0:
            y, X, trueBetas, featureNames, treatments, cy, ate = dgp(welfare, effect_type='homogeneous')
        else:
            y, X, trueBetas, featureNames, treatments, cy, ate = dgp(welfare, heterogeneous_select=heterogeneity)

        est, X_test = estimate_cf(y, X, treatments)

        estHLs.append(est)
        trueATEHLs.append(ate)
        evalSetHLs.append(X_test)

    bias, rmse, coverage, interval, lbs, ubs, ates = evalStats(estHLs, trueATEHLs, evalSetHLs)
    
    biasHLs[idx] = bias
    rmseHLs[idx] = rmse
    coverageHLs[idx] = coverage
    intervalHLs[idx] = interval

In [20]:
import joblib
joblib.dump(biasHLs, 'Results/biasHLs.pkl')
joblib.dump(rmseHLs, 'Results/rmseHLs.pkl')
joblib.dump(coverageHLs, 'Results/coverageHLs.pkl')
joblib.dump(intervalHLs, 'Results/intervalHLs.pkl')

['Results/intervalHLs.pkl']

In [27]:
biasHLs

array([ 2.02776808e-03,  3.94568975e-01,  3.43460337e-01, -2.61791121e+00])

In [28]:
rmseHLs

array([ 0.41843396,  0.56840136,  0.59779396, 40.88717615])

In [29]:
coverageHLs

array([1., 1., 1., 1.])

In [30]:
intervalHLs

array([ 22.07181591,  22.34502851,  24.49069995, 261.84933596])

## (6) Treatment Type (Continuous vs. Discrete)

In [16]:
K = 10
treatmentTypes = ['continuous', 'binary']

biasTTs = np.zeros(len(treatmentTypes))
rmseTTs = np.zeros(len(treatmentTypes))
coverageTTs = np.zeros(len(treatmentTypes))
intervalTTs = np.zeros(len(treatmentTypes))

for idx, treatment_type in enumerate(treatmentTypes):
    estTTs = []
    trueATETTs = []
    evalSetTTs = []
    criterion = 'het' if treatment_type == "binary" else 'mse'
    discrete_treatment = True if treatment_type == "binary" else False
    for i in tqdm(range(K)):
        y, X, trueBetas, featureNames, treatments, cy, ate = dgp(welfare, treatment_type=treatment_type)
        
        est, X_test = estimate_cf(y, X, treatments, criterion=criterion, discrete_treatment=discrete_treatment)

        estTTs.append(est)
        trueATETTs.append(ate)
        evalSetTTs.append(X_test)

    bias, rmse, coverage, interval, lbs, ubs, ates = evalStats(estTTs, trueATETTs, evalSetTTs)
    
    biasTTs[idx] = bias
    rmseTTs[idx] = rmse
    coverageTTs[idx] = coverage
    intervalTTs[idx] = interval

In [21]:
import joblib
joblib.dump(biasTTs, 'Results/biasTTs.pkl')
joblib.dump(rmseTTs, 'Results/rmseTTs.pkl')
joblib.dump(coverageTTs, 'Results/coverageTTs.pkl')
joblib.dump(intervalTTs, 'Results/intervalTTs.pkl')

['Results/intervalTTs.pkl']

In [23]:
biasTTs

array([-2.60607968e+01, -5.01181235e-03])

In [24]:
rmseTTs

array([26.40698023,  0.42194554])

In [25]:
coverageTTs

array([0.4, 1. ])

In [26]:
intervalTTs

array([51.24143738, 20.75211142])

## (7) Alignment

Below is to implement / have running state of variables

In [10]:
np.random.seed(123)
effect_type="heterogeneous"
effect_homogeneous=10
effect_heterogeneous=2
# treatment_type="continuous"
treatment_type="binary"
treatment_probability=0.5
heterogeneous_select=4
order=3
linearity="med"
N=1000
overlap=True
overlap_percent=0.5

featureNames = list(welfare.columns)

importantFeatureNames = ['wrkstat', 'race', 'year', 'hrs1', 'income', 'occ80', 'id', 'educ'] # top 8 important features based on Shapley visualization from 
importantFeatureIndices = []
for name in importantFeatureNames:
    importantFeatureIndices.append(featureNames.index(name)) 

# error terms
error = np.random.normal(size=(N,1))

# data generation
cov = welfare.cov()
means = welfare.mean(axis=0)
X = np.random.multivariate_normal(means.values, cov, size=N, check_valid='warn', tol=1e-8)

# linearity specification
if linearity != 'full':
    select = 0 # select n most important features for interactions and polynomials
    poly = PolynomialFeatures(degree=order, interaction_only=False, include_bias=False)

    if linearity == "high": 
        select = 2
    elif linearity == "med": 
        select = 4
    elif linearity == "low": 
        select = 8
    else: # if some typo, assume baseline of high
        select = 2

    poly.fit(X[:, importantFeatureIndices[:select]])
    fullData = poly.transform(X[:, importantFeatureIndices[:select]])
    fullNames = poly.get_feature_names(input_features=importantFeatureNames[:select])
    higherData = fullData[:, -(fullData.shape[1] - select):] # select only higher order
    higherNames = fullNames[-(len(fullNames) - select):]

    X = np.append(X, higherData, axis=1) 
    featureNames.extend(list(higherNames))

# treatment type
if treatment_type == "binary":
    # randomly assigned treatments with propensity treatment_probability
    treatments = np.random.choice([0, 1], size=N, p=[1 - treatment_probability, treatment_probability]).reshape((-1, 1))
    if not overlap:
        forced = random.sample(list(np.arange(treatments.shape[0])), int(treatments.shape[0] * overlap_percent))
        treatments[forced] = 0
    treated = treatments > 0
    
    # generate counterfactual treatment indicator vector
    c_treatments = deepcopy(treatments)
    c_treatments[treatments == 0] = 1
    c_treatments[treatments == 1] = 0

else:
    treatments = np.random.uniform(size=(N, 1))
    if not overlap:
        forced = random.sample(list(np.arange(treatments.shape[0])), int(treatments.shape[0] * overlap_percent))
        treatments[forced] = 0
    treated = treatments > 0.5
    c_treatments = deepcopy(treatments)
    # to-do
    c_treatments[treatments > 0] = 0
    c_treatments[treatments == 0] = treatments.mean()
    
# treatment effect
if effect_type == "homogeneous":

    T = treatments*effect_homogeneous
    CT = c_treatments*effect_homogeneous
    
else: # heterogeneous
    # heterogeneous treatment is effect 1 + effect * (sum of first heterogeneous_select important variables)
    heterogeneousIndices = importantFeatureIndices[:heterogeneous_select]
    
    T = 1 + (2*(X[:, heterogeneousIndices].sum(axis=1)))*treatments.ravel()
    T = T.reshape(-1, 1)
    T[~treated] = np.zeros((~treated).sum())
    
    CT = 1 + (2*(X[:, heterogeneousIndices].sum(axis=1)))*c_treatments.ravel()
    CT = CT.reshape(-1, 1)
    CT[treated] = np.zeros((treated).sum()) # everyone that was treated goes to untreated in counterfactual
    

betas = np.random.normal(size=X.shape[1]).reshape(-1,1)
y = T + X@betas + error
cy = CT + X@betas + error

In [184]:
X[:, heterogeneousIndices].mean(axis=0).sum() # average across the used columns to check ATE

15.720148792484203

In [176]:
tmp = y[treated] - cy[treated] # the treated
untmp = cy[~treated] - y[~treated] # the untreated
ate = (tmp.mean() + untmp.mean())/2

32.43845628547201

In [11]:
y, X, trueBetas, featureNames, treatments, cy, ate = dgp(welfare, N=36501, order=3, treatment_probability=0.5)

In [13]:
est, X_test = estimate_cf(y, X, treatments)